### Information Retrieval   |    Assignment 1-B

Step 1:

In [408]:
import pandas as pd
import numpy as np
import sys
import os
import random as rnd
import time
from collections import defaultdict

In [43]:
df_raw = pd.read_table('YandexRelPredChallenge.txt', delimiter="\t",index_col=None, 
                 header=0,names=['SessionID', 'TimePassed','TypeOfAction', 'QueryID','RegionID', 'ListOfURLs', 'URLID',
                                 '1','2','3','4','5',"6",'7','8'])
df_raw

In [2]:
# Open file    
fileHandler = open ('YandexRelPredChallenge.txt', "r")
# Get list of all lines in file
listOfLines = fileHandler.readlines()
# Close file 
fileHandler.close()
#Empty frame to fill in
index=[x for x in range(0,100000)]
df = pd.DataFrame(index=index, columns=['SessionID', 'TimePassed',
                 'TypeOfAction', 'QueryID','RegionID', 'ListOfURLs', 'URLID'])
df.name='data'

# Iterate over the lines and fill in the dataframe
for ind, line in zip(index, listOfLines):
#     print('index')
    line = line.strip().split()
    if line[2]=='Q':

        df.loc[ind] = pd.Series({'SessionID':line[0], 'TimePassed':line[1], 'TypeOfAction':line[2],
                                     'QueryID':line[3],'RegionID':line[4], 'ListOfURLs':line[5:], 'URLID':np.nan})
    else:
        df.loc[ind] = pd.Series({'SessionID':line[0], 'TimePassed':line[1], 'TypeOfAction':line[2],
                                     'QueryID':np.nan,'RegionID':np.nan, 'ListOfURLs':np.nan, 'URLID':line[3:]})

In [3]:
# Getting the types right. 
df.SessionID = df.SessionID.astype(int)
df.TimePassed = df.TimePassed.astype(int)
df.TypeOfAction = df.TypeOfAction.astype(str)
df.RegionID = df.RegionID.astype(float)
df.TimePassed = df.TimePassed.astype(float)
def parse_ulrid(x):
    if type(x)==list:
        return x[0]
    else:
        return x
df.URLID = df.URLID.apply(parse_ulrid)
df.URLID = df.URLID.astype(float)

Create new relevance:

So lets say that our experimental system is what Yandex return as a list now. Production or the system that is going to 'suck' (sort of a dumb system that we are going to compare against) will be constructed as shuffling each ranking for a query. 

Now, other approach, probably the better one: take the initial order as the 'benchmark' create only list of 0,1 (relevnace scores) for each system.
    - rember that we are interested only in three [3] first results.

In [34]:
# Create the relevance score ranking (those tables = that we called 'runs')
def relevance_list(l,acc=0.5, res_length=3):
    """ Control the 'accuracy'/'performance' with acc param. Only lists of length 3. """
    if type(l)==list:
        res = list()
        for x in l:
            if rnd.random()<acc:
                res.append((x, 1))
            else:
                res.append((x, 0))
        return rnd.sample(res, res_length)
    else:
        return l

df['id_rel_e'] = df.ListOfURLs.apply(relevance_list, acc=0.7) 
df['id_rel_p'] = df.ListOfURLs.apply(relevance_list) 

In [35]:
def extract_rel(l):
    if type(l) == list:
        res = list()
        for tup in l:
            res.append(tup[1])
        return res
    else:
        return l
def extract_id(l):
    if type(l) == list:
        res = list()
        for tup in l:
            res.append(tup[0])
        return res
    else:
        return l
df['rel_e'] = df.id_rel_e.apply(extract_rel)
df['rel_p'] = df.id_rel_p.apply(extract_rel)
df['id_e'] = df.id_rel_e.apply(extract_id)
df['id_p'] = df.id_rel_p.apply(extract_id)

Step 2: Calculate ERR

see the formuala for ERR at: http://olivier.chapelle.cc/pub/err.pdf

In [36]:
def ERR(l):
    """ List or relevance scores. """ 
    if type(l)==list:
        theta = [ (2**i - 1)/(2) for i in [0,1]]
        err=0
        for rank in range(len(l)):
            prod = 1
            for idx in range(rank):
                prod*=(1-theta[l[idx]])
            prod*=(1/(rank+1))*theta[l[rank]]
            err+=prod
        return err
    else:
        return l
df['err_e'] = df.rel_e.apply(ERR)
df['err_p'] = df.rel_p.apply(ERR)

Get the deltas

In [40]:
# Test to see if ERR calculation is implemented correctly and if E outperfroms P 
import math 
df['diff_e_p'] = df.err_e - df.err_p
def count_t(x):
    if not math.isnan(x):
        if x>0:
            return 1
        else:
            return 0 
    else:
        return x 
df['temp_count'] = df.diff_e_p.apply(count_t)
df.loc[:,['diff_e_p', 'temp_count']]
df.temp_count.value_counts()
df.drop(columns=['diff_e_p','temp_count'],inplace=True)

### Step 4.1: Estimating the params of click models: Random Click Model

In [179]:
# Get all the doc ids
data = df.drop(columns=['id_rel_e', 'id_rel_p', 'rel_e', 'rel_p', 'id_e',
       'id_p', 'err_e', 'err_p'])
data.QueryID = data.QueryID.astype(float)
list_doc_ids_raw = data.ListOfURLs[df.ListOfURLs.isnull()!=True].values
res = list()
for i in list_doc_ids_raw:
    res += i
all_doc_ids = res.copy()
# Get all the clicks
list_clicks_ids = data.URLID[df.URLID.isnull()!=True].values

# rho = probability of a doc to be clicked.
rho = len(list_clicks_id)/len(all_doc_ids)
print(rho)

0.13445559411047547


### Step 4.2: Estimating the params of click models: Posistion-based Model

Prepare the data for training
    - assumption: we assume that all the queries have at least 3 docs shown.
    
    TODO: check if true

In [613]:
# Create the tables with all the params
n_docs = 3
unique_queries = data.QueryID[data.QueryID.isnull()!=True].astype(int).unique()
n_queries = len(unique_queries) 
gammas = np.random.rand(n_docs)
alphas = np.random.rand(n_docs*n_queries).reshape((n_docs,n_queries))

In [544]:
start = time.time()

# Create query_doc empty pairs
query_doc_pairs = defaultdict()
for i in unique_queries:
    query_doc_pairs[i]=[]
    
# This code will give an error at the very last session ! -> move last session to the different bit.
sessions = data.SessionID.unique()[:-1]
for session in sessions:
    data_session = data[data.SessionID==session]
    click_ids =  data_session.URLID[data_session.URLID.isnull()!=True].values
    query_ids = data_session.QueryID[data_session.QueryID.isnull()!=True].values
    for query_id in query_ids:
        first_3_docs = data_session.ListOfURLs[data.QueryID==query_id].iloc[0]
        first_3_docs = np.array(list(map(int,first_3_docs))[:3])
        # take index in the current data_session
        query_id_ind = data_session.QueryID[data_session.QueryID==query_id].index[0]
        flag = not math.isnan(data.URLID.iloc[query_id_ind+1])
        # if the next entry is click
        if flag:
            click_log = np.array([0,0,0])
            query_id_ind+=1
            while flag==1:
                click_urlid = data.URLID.iloc[query_id_ind]
                # if click_urlid in first_3_docs then click is 1 on that position.
                doc_u_id = np.where(first_3_docs==click_urlid)
                click_log[doc_u_id]=1
                # increase the count
                query_id_ind +=1
                flag = not math.isnan(data.URLID.iloc[query_id_ind])
            # do updates
        else:
            click_log = np.array([0,0,0])
            # do updates
            
        # Save query_id-click_log (=session-doc) pair for each query_id
        query_doc_pairs[query_id].append(click_log)
    
end = time.time()
print(f'suka bleyat : seconds spent {end - start}, mins spent {(end - start)/60}')

seconds spent 139.9012987613678, mins spent 2.3316883126894634


In [513]:
query_doc_pairs_old = query_doc_pairs.copy()
# query_doc_pairs_old

Perform EM steps (update assignments ; update params):
    
    1) initialize alpha and gamma at random 
    2) For each time step:
        - Expecation: do not really have to do this step [?], what we want to see at the end: P(C|u) 
            - calculate P(C|u) for each u (u is from 1 to 3 = positions in the shown list)
            - " P(C|u) = P(A_uq|E,u)P(E|u) "
                
        - Maximization: update alpha and gamma params
        


### Alphas 

In [615]:
assert unique_queries.shape[0] == alphas.shape[1], 'Error, bleayt'
# for comparison :
alphas_old = alphas.copy()

In [625]:
start = time.time()

# Do the updates for alphas:

# refer to alphas array with query_num as that it is how the are strored there:
for query_num, query_id in enumerate(query_doc_pairs):
    for doc_u in range(n_docs):
        factor_1 = 1/len(query_doc_pairs[query_id])
        summ =  0
        for sessions_docs in query_doc_pairs[query_id]:
            click = sessions_docs[doc_u]
            summ += click + (1-click)*( (1-gammas[doc_u])*alphas[doc_u][query_num] ) / \
                                        (1-gammas[doc_u]*alphas[doc_u][query_num])
        alphas[doc_u][query_num] = factor_1*summ
        
end = time.time()
print(f'seconds spent {end - start}, mins spent {(end - start)/60}')

seconds spent 1.3521900177001953, mins spent 0.022536500295003255


In [627]:
# is that true that we ve changed every alpha ?
not (alphas_old == alphas).any()

True

### Gammas

In [629]:
gammas

array([0.64732249, 0.79368901, 0.802856  ])

In [ ]:
start = time.time()

# Do the updates for gammas:

# refer to alphas array with query_num as that it is how the are strored there:
for query_num, query_id in enumerate(query_doc_pairs):
    for doc_u in range(n_docs):
        factor_1 = 1 / len(query_doc_pairs[query_id])
        summ =  0
        for sessions_docs in query_doc_pairs[query_id]:
            click = sessions_docs[doc_u]
            summ += click + (1-click)*( (1-gammas[doc_u])*alphas[doc_u][query_num] ) / \
                                        (1-gammas[doc_u]*alphas[doc_u][query_num])
        alphas[doc_u][query_num] = factor_1*summ
        
end = time.time()
print(f'seconds spent {end - start}, mins spent {(end - start)/60}')